# e-Commerce EDA

useful links:
- https://userpilot.com/blog/customer-growth/
- https://chartio.com/learn/product-analytics/top-product-metrics/


This week you'll work on the e-commerce EDA using Python. **You will use a Python SQL connector to perform queries on your newly created database.**

This is your third EDA, so we are confident you can do this easily. This time also, you will create a report at the end of the project that will be presented to the management team of 'VS Group.' They would like to see how much you have understood the e-commerce business and your data analysis keeping in mind all the KPIs.



## Epic 1: Exploratory Data Analysis
There are plenty of insights you can deduce from this dataset. Try to look for some these insights yourself. As a starting point, here are some ideas that you could present to the team:

- What customer purchasing patterns can you deduce? Such as during the days of the week, weekly, monthly, quarterly, yearly, etc.

- Are there specific days/months/quarters when the sales have been unusually high/low, and what could be the possible reasons? How about the profit and loss margin?

- Which states and which customers made the highest number of orders? Are they the same as the highest spending states and customers?

- Can you make a map showing the 5 states generating the most and least sales revenue?

- Can we see the quarterly revenue behavior?

- Can you create a plot showing the growth rate of new customers over the months?

- What do you think about the customers? Are they individuals or wholesalers? Why would you say so?

- Are there any issues with the dataset?

**You don't have to create all these plots. Choose the most relevant ones for your analysis.**

**Optional: make a small selection of the plots and try to present them as Plotly-Express animations. Please remember that the most important thing is to make a good analysis, independent of which library you use to create your graphs.**


In [12]:
# Imoports
import pandas as pd
import sqlite3
import plotly.express as px

## Load data set

In [13]:
# Connect SQLite database.
db_conn = sqlite3.connect("SuperstoreDB/superstore.db")

An initial eda and data cleaning has been performed in superstore_db_preperartion.ipynb, so there is no need anymore to check for missing or duplicated data, etc.

## What customer purchasing patterns can be deduced?

In [14]:
# Unique customers per year.
customers_per_year = pd.read_sql(
    """
    SELECT Year, COUNT (DISTINCT CustomerID) AS CustomerCount
    FROM (
        SELECT
            *,
            SUBSTR(OrderDate, 1, 4) AS Year
        FROM Orders
    )
    GROUP BY Year

    """, db_conn)

fig = px.bar(customers_per_year, x='Year', y='CustomerCount')
fig.update_layout(title='Customer Count Per Year')
fig.show()

**Observation**: The amount of active users is slightly increasing over the years with a small dip in 2015. We define a user as active if they placed at least one order in a given year.

In [24]:
# Orders per year.
orders_yearly = pd.read_sql(
    """
    SELECT Year, COUNT (DISTINCT OrderID) AS OrderCount
    FROM (
        SELECT
            *,
            SUBSTR(OrderDate, 1, 4) AS Year
        FROM Orders
    )
    GROUP BY Year

    """, db_conn)

# Calculate year-on-year percentage growth
orders_yearly['YearOnYearGrowth'] = orders_yearly['OrderCount'].pct_change() * 100

In [25]:
fig = px.bar(orders_yearly, x='Year', y='OrderCount')
fig.update_layout(title='Orders Yearly')
fig.show()

**Observation**: The ammount of orders per year has been growing steadily between 2014 and 2017. When compared to the plot above showing 'Customer Count Per Year', we can see that the order count per year is growing quicker.

In [26]:
# Year-on-year growth of order numbers in percentage.
fig = px.bar(orders_yearly, x='Year', y='YearOnYearGrowth', title='Year-on-Year Order Growth (%)')
fig.show()

**Observation**: By checking the year-on-year growth of order amount, we can see that company is growing each year more than the last one. So we could say the growth is exponential. this is very good news, especially when we compare those numbers to the relatively stagnant customer count per year. We could conclude that even though our customer base is not growing fast, their demand still is.

In [40]:
# Orders per month.
orders_monthly = pd.read_sql(
    """
    SELECT Month, COUNT (DISTINCT OrderID) AS OrderCount
    FROM (
        SELECT
            *,
            SUBSTR(OrderDate, 6, 2) AS Month
        FROM Orders
    )
    GROUP BY Month

    """, db_conn)

fig = px.bar(orders_monthly, x='Month', y='OrderCount')
fig.update_layout(title='Orders Monthly')
fig.show()

In [41]:
# Orders per day of month.
orders_per_day_of_month = pd.read_sql(
    """
    SELECT Day, COUNT (DISTINCT OrderID) AS OrderCount
    FROM (
        SELECT
            *,
            SUBSTR(OrderDate, 9, 2) AS Day
        FROM Orders
    )
    GROUP BY Day

    """, db_conn)

fig = px.bar(orders_per_day_of_month, x='Day', y='OrderCount')
fig.update_layout(title='Orders per day of month')
fig.show()

In [42]:
# Orders per month.
orders_monthly_over_years = pd.read_sql(
    """
    SELECT
    SUBSTR(OrderDate, 1, 4) AS Year,
    SUBSTR(OrderDate, 6, 2) AS Month,
    COUNT(OrderID) AS OrderCount
    FROM Orders
    GROUP BY Year, Month
    """, db_conn)

fig = px.line(orders_monthly_over_years, x='Month', y='OrderCount', color='Year')
fig.update_layout(title='Orders Monthly')
fig.show()


In [43]:
# Monthly Sales
sales_monthly_over_years = pd.read_sql(
    """
    SELECT
    SUBSTR(OrderDate, 1, 4) AS Year,
    SUBSTR(OrderDate, 6, 2) AS Month,
    SUM(Sales) / 100.0 AS Sales
    FROM Orders
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Year, Month
    """, db_conn)

fig = px.line(sales_monthly_over_years, x='Month', y='Sales', color='Year')
fig.update_layout(title='Sales Monthly')
fig.show()

In [44]:
# Monthly Profit
sales_monthly_over_years = pd.read_sql(
    """
    SELECT
    SUBSTR(OrderDate, 1, 4) AS Year,
    SUBSTR(OrderDate, 6, 2) AS Month,
    SUM(Profit) / 100.0 AS Profit 
    FROM Orders
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Year, Month
    """, db_conn)

fig = px.line(sales_monthly_over_years, x='Month', y='Profit', color='Year')
fig.update_layout(title='Profit Monthly')
fig.show()

In [45]:
# Which states and which customers made the highest number of orders?
# Are they the same as the highest spending states and customers?

In [46]:
# Which states made the highest number of orders?
orders_per_state = pd.read_sql(
    """
    SELECT
    Region,
    State,
    COUNT(*) AS "Orders"
    FROM Orders
    JOIN Addresses
    ON Orders.AddressID = Addresses.AddressID
    GROUP BY Region, State
    """, db_conn)

fig = px.sunburst(orders_per_state, path=['Region', 'State'], values='Orders')
fig.update_layout(title='Number of Orders by State')
fig.show()

In [47]:
# Which states made the highest sales?
sales_per_state = pd.read_sql(
    """
    SELECT
    Region,
    State,
    SUM(Sales)/100.0 AS "Sales ($)"
    FROM Orders
    JOIN Addresses
    ON Orders.AddressID = Addresses.AddressID
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Region, State
    """, db_conn)

fig = px.sunburst(sales_per_state, path=['Region', 'State'], values='Sales ($)')
fig.update_layout(title='Sales by State')
fig.show()

In [48]:
# Which states made the highest profit?
profit_per_state = pd.read_sql(
    """
    SELECT
    Region,
    State,
    SUM(Profit)/100.0 AS "Profit ($)"
    FROM Orders
    JOIN Addresses
    ON Orders.AddressID = Addresses.AddressID
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Region, State
    ORDER BY "Profit ($)"
    """, db_conn)

#profit_per_state
fig = px.bar(profit_per_state, x='State', y='Profit ($)')
fig.update_layout(title='Profit by State')
fig.show()

In [49]:
# Which customers made the highest number of orders?
orders_per_customer_segment = pd.read_sql(
    """
    SELECT
    Segment,
    COUNT(*) AS "Orders"
    FROM Orders
    JOIN Customers
    ON Orders.CustomerID = Customers.CustomerID
    GROUP BY Segment
    """, db_conn)

fig = px.pie(orders_per_customer_segment, values='Orders', names='Segment')
fig.update_layout(title='Orders by Customer Segment')
fig.show()

In [50]:
# Which customers made the highest sales?
sales_per_customer_segment = pd.read_sql(
    """
    SELECT
    Segment,
    SUM(Sales)/100.0 AS "Sales ($)"
    FROM Orders
    JOIN Customers
    ON Orders.CustomerID = Customers.CustomerID
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Segment
    """, db_conn)

sales_per_customer_segment
fig = px.pie(sales_per_customer_segment, values="Sales ($)", names='Segment')
fig.update_layout(title='Sales by Customer Segment')
fig.show()

In [51]:
# Which customers made the highest profit?
profit_per_customer_segment = pd.read_sql(
    """
    SELECT
    Segment,
    SUM(Profit)/100.0 AS "Profit ($)"
    FROM Orders
    JOIN Customers
    ON Orders.CustomerID = Customers.CustomerID
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Segment
    """, db_conn)

fig = px.pie(profit_per_customer_segment, values="Profit ($)", names='Segment')
fig.update_layout(title='Profit by Customer Segment')
fig.show()

# Can you make a map showing the 5 states generating the most and least sales revenue?

In [52]:
# Can you make a map showing the 5 states generating the most and least sales revenue?
profit_per_state = pd.read_sql(
    """
    SELECT
    State,
    SUM(Profit)/100.0 AS "Profit ($)"
    FROM Orders
    JOIN Addresses
    ON Orders.AddressID = Addresses.AddressID
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY State
    ORDER BY "Profit ($)"
    """, db_conn)

profit_per_state.head(5)

,State,Profit ($)
0,Texas,-25729.42
1,Ohio,-16971.61
2,Pennsylvania,-15560.03
3,Illinois,-12607.79
4,North Carolina,-7491.59


In [53]:
# Load the US codes required by the px.choropleth map.
us_codes = pd.read_json("states_titlecase.json")

# Merge them into our data.
profit_per_state = profit_per_state.merge(us_codes, left_on='State', right_on='name', how='left')
profit_per_state.head()

,State,Profit ($),name,abbreviation
0,Texas,-25729.42,Texas,TX
1,Ohio,-16971.61,Ohio,OH
2,Pennsylvania,-15560.03,Pennsylvania,PA
3,Illinois,-12607.79,Illinois,IL
4,North Carolina,-7491.59,North Carolina,NC


In [54]:
fig = px.choropleth(
    profit_per_state,
    scope="usa",
    locations='abbreviation',  # Use 'State' column as location
    locationmode='USA-states',  # Set location mode to USA states
    color='Profit ($)',  # Color based on profit
    color_continuous_scale='RdYlGn',  # Choose color scale
    labels={'Profit ($)': 'Profit ($)'},  # Label for color bar
    title='Profit by State'  # Title of the plot
)

# Show the plot
fig.show()

In [55]:
# Sort the profit_per_state DataFrame by "Profit ($)" column
profit_per_state_sorted = profit_per_state.sort_values(by="Profit ($)", ascending=False)

# Get the top 5 and bottom 5 states based on sales revenue
top_5_states = profit_per_state_sorted.head(5)
bottom_5_states = profit_per_state_sorted.tail(5)

# Create a new column indicating whether each state is in the top or bottom group
profit_per_state_sorted['Group'] = ''  # Initialize the 'Group' column
profit_per_state_sorted.loc[top_5_states.index, 'Group'] = 'Top 5'
profit_per_state_sorted.loc[bottom_5_states.index, 'Group'] = 'Bottom 5'

# Filter the DataFrame to include only the top 5 and bottom 5 states
selected_states = profit_per_state_sorted[(profit_per_state_sorted['Group'] == 'Top 5') | (profit_per_state_sorted['Group'] == 'Bottom 5')]

# Create the choropleth map using Plotly Express
fig = px.choropleth(
    selected_states,
    scope="usa",
    locations='abbreviation',  # Use 'State' column as location
    locationmode='USA-states',  # Set location mode to USA states
    color='Group',  # Color based on the group column
    color_discrete_map={'Top 5': 'green', 'Bottom 5': 'red'},  # Assign colors to groups
    labels={'Group': 'Group'},  # Label for color legend
    title='Top and Bottom 5 States by Profit',  # Title of the plot
    hover_data={'abbreviation': False, 'Profit ($)': ':,.2f', 'State': True}  # Customize hover information
)

# Show the plot
fig.show()


# Can we see the quarterly revenue behavior?

In [56]:
# Can we see the quarterly revenue behavior?


# Can you create a plot showing the growth rate of new customers over the months?

In [57]:
# Can you create a plot showing the growth rate of new customers over the months?
new_customers_per_month = pd.read_sql(
    """
    SELECT
    SUBSTR(FirstOrderDate, 1, 4) AS Year,
    SUBSTR(FirstOrderDate, 6, 2) AS Month,
    COUNT(CustomerID) AS NewCustomers
    FROM (
        SELECT
        Orders.CustomerID,
        MIN(OrderDate) AS FirstOrderDate
        FROM Orders
        JOIN Customers
        ON Orders.CustomerID = Customers.CustomerID
        JOIN OrdersDetails
        ON Orders.OrderID = OrdersDetails.OrderID
        GROUP BY Orders.CustomerID
    )
    GROUP BY Year, Month
    """, db_conn)

new_customers_per_month

# Plot the growth rate over the months
fig = px.line(new_customers_per_month, x='Month', y='NewCustomers', color='Year')
fig.show()


In [58]:
# Can you create a plot showing the growth rate of new customers over the months?
new_customers_per_month = pd.read_sql(
    """
    SELECT
    SUBSTR(FirstOrderDate, 1, 7) AS YearMonth,
    COUNT(CustomerID) AS NewCustomers
    FROM (
        SELECT
        Orders.CustomerID,
        MIN(OrderDate) AS FirstOrderDate
        FROM Orders
        JOIN Customers
        ON Orders.CustomerID = Customers.CustomerID
        JOIN OrdersDetails
        ON Orders.OrderID = OrdersDetails.OrderID
        GROUP BY Orders.CustomerID
    )
    GROUP BY YearMonth
    """, db_conn)

# Calculate cumulative sum of new customers
new_customers_per_month['CumulativeNewCustomers'] = new_customers_per_month['NewCustomers'].cumsum()

# Calculate monthly growth rate of customers
new_customers_per_month['NewCustomerGrowthRate'] = (new_customers_per_month['NewCustomers']/new_customers_per_month['CumulativeNewCustomers']) * 100

# Split YearMonth column into Year and Month
new_customers_per_month[['Year', 'Month']] = new_customers_per_month['YearMonth'].str.split('-', expand=True)

# Plot the growth rate of new customers over the months
fig = px.line(new_customers_per_month, x='YearMonth', y='NewCustomers', title='Growth Rate of New Customers Over Time')
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Number of New Customers')
fig.show()

In [59]:
# Plot the growth of new customers over the months
fig = px.line(new_customers_per_month, x='YearMonth', y='CumulativeNewCustomers', title='Growth of New Customers Over Time')
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Cumulative Number of New Customers')
fig.show()

In [60]:
fig = px.line(new_customers_per_month, x='Month', y='NewCustomerGrowthRate', color='Year', title='Customer Growth Rate')
fig.update_yaxes(title='Customer Growt Rate (%)')
fig.show()

# What do you think about the customers? Are they individuals or wholesalers? Why would you say so?

In [61]:
# What do you think about the customers? Are they individuals or wholesalers? Why would you say so?
# Which customers made the highest profit?
customers_df = pd.read_sql(
    """
    SELECT
    Customers.CustomerID,
    Segment,
    COUNT(*) AS OrderCount,
    SUM(Quantity) AS OrderQuantity,
    SUM(Profit) / 100.0 AS "Profit ($)",
    SUM(Sales) / 100.0 AS "Sales ($)"
    FROM Orders
    JOIN Customers
    ON Orders.CustomerID = Customers.CustomerID
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    GROUP BY Customers.CustomerID
    """, db_conn)

In [62]:
customers_df

,CustomerID,Segment,OrderCount,OrderQuantity,Profit ($),Sales ($)
0,AA-10315,Consumer,11,30,-362.93,5563.54
1,AA-10375,Consumer,15,41,277.31,1056.36
2,AA-10480,Consumer,12,36,435.77,1790.51
3,AA-10645,Consumer,18,64,857.77,5086.90
4,AB-10015,Consumer,6,13,129.33,886.15
...,...,...,...,...,...,...
788,XP-21865,Consumer,28,100,621.09,2374.62
789,YC-21895,Corporate,8,31,1305.58,5454.34
790,YS-21880,Corporate,12,58,1778.23,6720.43
791,ZC-21910,Consumer,31,105,-1032.19,8025.63


In [63]:
for i in ['OrderCount', 'OrderQuantity', 'Profit ($)', 'Sales ($)']:

    # Calculate the average y value
    average_y = customers_df[i].mean()

    # Plot the bar chart
    fig = px.bar(customers_df.sort_values(i), x='CustomerID', y=i, color='Segment')

    # Add a horizontal line for the average y value
    fig.add_hline(y=average_y, line_dash="dot", annotation_text=f'Average {i}: {average_y:.2f}', annotation_position="bottom right")

    # Show the plot
    fig.show()

# Average Order Value

In [76]:
# AverageOrderValue = Revenue/TotalOrders

aov = pd.read_sql(
    """
    SELECT
    SUM(Sales) / COUNT(*) / 100.0 AS AverageOrderValue
    FROM Orders
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    JOIN Products
    ON OrdersDetails.ProductID = Products.ProductID
    """, db_conn)

In [77]:
aov

,AverageOrderValue
0,230.03


In [78]:
monthly_aov = pd.read_sql(
    """
    SELECT
    SUBSTR(OrderDate, 1, 4) AS Year,
    SUBSTR(OrderDate, 6, 2) AS Month,
    SUM(Sales) / COUNT(*) / 100.0 AS AverageOrderValue
    FROM Orders
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    JOIN Products
    ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Month, Year
    """, db_conn)

In [150]:
fig = px.line(monthly_aov, x='Month', y='AverageOrderValue', color='Year', title='Monthly Average Order Value')
fig.show()

# Average Profit Margin

In [112]:
def monthly_average_profit_margin(year):
    # Monthly Sales and Profit
    average_profit_margin = pd.read_sql(
        f"""
        SELECT
        SUBSTR(OrderDate, 6, 2) AS Month,
        SUM(Sales) / 100.0 AS Sales,
        SUM(Profit) / 100.0 AS Profit
        FROM Orders
        JOIN OrdersDetails
        ON Orders.OrderID = OrdersDetails.OrderID
        WHERE SUBSTR(OrderDate, 1, 4) = '{year}'
        GROUP BY Month
        """, db_conn)
    
    # Plot the line chart
    fig = px.line(average_profit_margin, x='Month', y='Profit', title=f'Profit and Sales for {year}', color_discrete_sequence=['green'])

    # Add the bar chart with y='Sales'
    fig.add_bar(x=average_profit_margin['Month'], y=average_profit_margin['Sales'], name='Sales', marker_color='lightblue')

    return fig



In [114]:
for year in [2014, 2015, 2016, 2017]:
    f = monthly_average_profit_margin(year)
    f.show()

# Annual Repurchase Rate

In [130]:
# Annual Repurchase Rate
annual_repurchase_rate = pd.read_sql(
        f"""
        SELECT
        Orders.CustomerID,
        COUNT(Orders.OrderID) AS OrderCount
        FROM Orders
        JOIN Customers
        ON Orders.CustomerID = Customers.CustomerID
        JOIN OrdersDetails
        ON Orders.OrderID = OrdersDetails.OrderID
        GROUP BY Orders.CustomerID
        
        
        """, db_conn)

annual_repurchase_rate

,CustomerID,OrderCount
0,AA-10315,11
1,AA-10375,15
2,AA-10480,12
3,AA-10645,18
4,AB-10015,6
...,...,...
788,XP-21865,28
789,YC-21895,8
790,YS-21880,12
791,ZC-21910,31


In [124]:
# Can you create a plot showing the growth rate of new customers over the months?
new_customers_per_month = pd.read_sql(
    """
    SELECT
    SUBSTR(FirstOrderDate, 1, 7) AS YearMonth,
    COUNT(CustomerID) AS NewCustomers
    FROM (
        SELECT
        Orders.CustomerID,
        MIN(OrderDate) AS FirstOrderDate
        FROM Orders
        JOIN Customers
        ON Orders.CustomerID = Customers.CustomerID
        JOIN OrdersDetails
        ON Orders.OrderID = OrdersDetails.OrderID
        GROUP BY Orders.CustomerID
    )
    GROUP BY YearMonth
    """, db_conn)

# Calculate cumulative sum of new customers
new_customers_per_month['CumulativeNewCustomers'] = new_customers_per_month['NewCustomers'].cumsum()

# Calculate monthly growth rate of customers
new_customers_per_month['NewCustomerGrowthRate'] = (new_customers_per_month['NewCustomers']/new_customers_per_month['CumulativeNewCustomers']) * 100

# Split YearMonth column into Year and Month
new_customers_per_month[['Year', 'Month']] = new_customers_per_month['YearMonth'].str.split('-', expand=True)

# Plot the growth rate of new customers over the months
fig = px.line(new_customers_per_month, x='YearMonth', y='NewCustomers', title='Growth Rate of New Customers Over Time')
fig.update_xaxes(title='Month')
fig.update_yaxes(title='Number of New Customers')
fig.show()

In [129]:
new_customers_per_month

,YearMonth,NewCustomers,CumulativeNewCustomers,NewCustomerGrowthRate,Year,Month
0,2014-01,32,32,100.000000,2014,01
1,2014-02,24,56,42.857143,2014,02
2,2014-03,65,121,53.719008,2014,03
3,2014-04,56,177,31.638418,2014,04
4,2014-05,56,233,24.034335,2014,05
5,2014-06,48,281,17.081851,2014,06
6,2014-07,44,325,13.538462,2014,07
7,2014-08,49,374,13.101604,2014,08
8,2014-09,68,442,15.384615,2014,09
9,2014-10,42,484,8.677686,2014,10


# Products

In [137]:
# 1. What is the category generating the maximum sales revenue?
pd.read_sql(
    """
    SELECT Category, SUM(Sales)/100.0 AS "Sales Revenue ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Category
    ORDER BY "Sales Revenue ($)" DESC;
    """, db_conn)

,Category,Sales Revenue ($)
0,Technology,836149.37
1,Furniture,741993.82
2,Office Supplies,719033.79


In [143]:
# Which Categories and Sub-Categories made the highest sales?
sales_per_category = pd.read_sql(
    """
    SELECT
    Category,
    SubCategory,
    SUM(Sales)/100.0 AS "Sales ($)"
    FROM OrdersDetails
    JOIN Products ON OrdersDetails.ProductID = Products.ProductID
    GROUP BY Category, SubCategory
    """, db_conn)

fig = px.sunburst(sales_per_category, path=['Category', 'SubCategory'], values='Sales ($)')
fig.update_layout(title='Sales by Categories')
fig.show()

In [131]:
products_df = pd.read_sql(
    """
    SELECT
    *
    FROM Orders
    JOIN OrdersDetails
    ON Orders.OrderID = OrdersDetails.OrderID
    JOIN Products
    ON OrdersDetails.ProductID = Products.ProductID
    """, db_conn)

In [132]:
products_df

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,AddressID,OrderID,ProductID,Sales,Quantity,Discount,Profit,ProductID,ProductName,Category,SubCategory
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,42420-000001,CA-2016-152156,FUR-BO-10001798,26195,2,0.00,4191,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,42420-000001,CA-2016-152156,FUR-CH-10000454,73194,3,0.00,21958,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,90036-000001,CA-2016-138688,OFF-LA-10000240,1462,2,0.00,687,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,33311-000001,US-2015-108966,FUR-TA-10000577,95757,5,0.45,-38303,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,33311-000001,US-2015-108966,OFF-ST-10000760,2236,2,0.20,251,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,33180-000001,CA-2014-110422,FUR-FU-10001889,2524,3,0.20,410,FUR-FU-10001889,Ultra Door Pull Handle,Furniture,Furnishings
9982,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,92627-000001,CA-2017-121258,FUR-FU-10000747,9196,2,0.00,1563,FUR-FU-10000747,Tenex B1-RE Series Chair Mats for Low Pile Car...,Furniture,Furnishings
9983,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,92627-000001,CA-2017-121258,TEC-PH-10003645,25857,2,0.20,1939,TEC-PH-10003645,Aastra 57i VoIP phone,Technology,Phones
9984,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,92627-000001,CA-2017-121258,OFF-PA-10004041,2960,4,0.00,1332,OFF-PA-10004041,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",Office Supplies,Paper
